In [1]:
# need to use updated master JupiterCVML code
import sys
# sys.path.append('/home/li.yu/code/JupiterCVML/europa/base/src/europa')

import os
os.environ["BRT_ENV"] = 'prod'
import json
import random
import brtdevkit
print(brtdevkit.__version__)
brtdevkit.log = 'info'
import os
os.environ['AWS_PROFILE'] = 'default'

import time
import numpy as np
import pandas as pd
from datetime import datetime, timezone

from brtdevkit.core.db.athena import AthenaClient, Table
from brtdevkit.data import Image, Dataset

# from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import *
# from aletheia_dataset_creator.config.dataset_config import *

from jupiterdata.config.dataset_config import *
from jupiterdata.utils.dataset import *

pd.set_option('display.max_columns', None)

# %matplotlib inline
%load_ext autoreload
%autoreload 2

7.14.2


/home/li.yu/anaconda3/envs/brtdevkit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/li.yu/anaconda3/envs/brtdevkit/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
import inspect
inspect.getmodule(Dataset)

<module 'brtdevkit.data.core.dataset' from '/home/li.yu/anaconda3/envs/brtdevkit/lib/python3.9/site-packages/brtdevkit/data/core/dataset.py'>

In [3]:
print(LEFT_CAMERAS)
print(ALL_CAMERA_PAIRS_LIST)

['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left', 'T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02']
[{'front-center-left': 'front-center-right', 'front-left-left': 'front-left-right', 'front-right-left': 'front-right-right', 'side-left-left': 'side-left-right', 'side-right-left': 'side-right-right', 'rear-left': 'rear-right', 'front-center-right': 'front-center-left', 'front-left-right': 'front-left-left', 'front-right-right': 'front-right-left', 'side-left-right': 'side-left-left', 'side-right-right': 'side-right-left', 'rear-right': 'rear-left'}, {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03', 'I02': 'I04'}, {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15', 'I02': 'I03'}]


In [5]:
# test_dataset = Dataset.retrieve(id='64ed2657926aefcd654e8269')
test_dataset = Dataset.retrieve(name='halo_vehicles_driving_through_dust_images_nodust_reserved')
print(test_dataset.id)
test_df = test_dataset.to_dataframe()
test_df.shape

65f0736152d8fb5d29fb7adb


2024-03-20 07:15:08,006 - APIRequestor - ERROR - API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}


(7598, 91)

In [6]:
test_df[['id', 'camera_location']].groupby('camera_location').count()

,id
camera_location,
T05,507
T06,505
T09,1975
T10,1981
T13,1314
T14,1316


In [2]:
datasets = [
    '20240119_halo_rgb_left_day_less_night_al_to_label',
    '20240119_halo_rgb_diversity_left_daytime',
    '20231219_halo_rgb_al_to_label',
    '20231219_halo_rgb_diversity_to_label',
    '20231206_halo_rgb_diversity_train_to_label_pt1',
    '20231206_halo_rgb_al_train_to_label_pt1',
    '20231206_halo_rgb_stereo_fps_dedup',
    '20231219_halo_rgb_stereo_fps_dedup',
]
test_dfs = []
for dataset in datasets:
    test_dataset = Dataset.retrieve(name=dataset)
    test_df = test_dataset.to_dataframe()
    print(dataset, test_df.shape)
    test_dfs.append(test_df)
pd.concat(test_dfs, ignore_index=True).to_csv('/data/jupiter/li.yu/data/halo_hard_cases/20231206_20231219_sent_for_labeling.csv', index=False)

20240119_halo_rgb_left_day_less_night_al_to_label (7457, 146)
20240119_halo_rgb_diversity_left_daytime (4128, 146)
20231219_halo_rgb_al_to_label (7552, 147)
20231219_halo_rgb_diversity_to_label (1375, 144)
20231206_halo_rgb_diversity_train_to_label_pt1 (4955, 157)
20231206_halo_rgb_al_train_to_label_pt1 (17049, 157)
20231206_halo_rgb_stereo_fps_dedup (15319, 158)
20231219_halo_rgb_stereo_fps_dedup (5584, 147)


## Run Athena Query

In [5]:
# hard_drive_name = 'JUPD-048_2022-09-27'
start_datetime  = datetime(2021, 12, 16, hour=22, minute=14, second=0) #, tzinfo=timezone.utc)
end_datetime  = datetime(2021, 12, 16, hour=22, minute=14, second=35) #, tzinfo=timezone.utc)
# start_datetime  = datetime(2022, 9, 28, tzinfo=timezone.utc) #, hour=3, minute=33, second=33) #, tzinfo=timezone.utc)
# start_datetime  = datetime(2023, 8, 28, hour=0, minute=0)   #, tzinfo=timezone.utc)
# end_datetime    = datetime(2023, 8, 30, hour=23, minute=59)  # this is EXclusive

database = "mesa-data-catalog-prod"
table = "image_jupiter"

# query = \
# f"""
# SELECT *
# FROM {table} T
# WHERE 
#     T.collected_on >= cast('{start_datetime}' as timestamp)
#     AND T.collected_on < cast('{end_datetime}' as timestamp)
#     AND T.robot_name = 'loamy_733'
# """

query = \
f"""
SELECT *
FROM {table} T
WHERE 
    T.hard_drive_name = 'JUPD-0306_2024-2-28'
"""

# query = """
# SELECT id, image, properties__json FROM annotation_jupiter
# WHERE style = 'categorical'
# AND created_at > date('2024-01-01')
# AND state in ('review', 'ok')
# AND (properties__json LIKE '%"title": "Pose"%' OR properties__json LIKE '%"title": "Human Clothing"%' OR properties__json LIKE '%"title": "Human Occlusion"%') 
# AND kind = 'labelbox'
# """

# """
# SELECT
#         *
#     FROM annotation_jupiter
#     WHERE
#      annotation_jupiter.image IN ('642607df1620247e2285d6ac')
# """

# """
# SELECT id, camera_location, operation_time, collected_on, state, county, operating_field_name, hdr_mode, robot_name, special_notes
# T.collected_on >= cast('{start_datetime}' as timestamp)
#     AND T.collected_on < cast('{end_datetime}' as timestamp)
#     AND T.camera_location LIKE 'front-%'
#     AND T.camera_location LIKE '%-left'
#     AND T.hard_drive_name = 'JUPD-154_2023-01-14'
#     AND T.robot_name LIKE 'loamy%'
#     AND T.operation_time != 'daytime'
#     AND T.has_nearby_stop_event = true
# ORDER BY RAND()
# LIMIT 40000
# """

start = time.time()
athena = AthenaClient()
df = athena.get_df(query)
end = time.time()
print(end - start, 's')
df.shape, len(df) // 2

## Run Databricks Query

In [8]:
from jupiterdata.db import databricks_connector

databricks = databricks_connector.Databricks()
# note on json extraction https://docs.databricks.com/en/sql/language-manual/sql-ref-json-path-expression.html

query = f"""
SELECT
  mesa_prod.mesa_lake_prod.image_jupiter.id AS id,
  mesa_prod.mesa_lake_prod.image_jupiter.has_human_annotation AS has_human_annotation,
  mesa_prod.mesa_lake_prod.image_jupiter.operation_time AS operation_time,
  mesa_prod.mesa_lake_prod.image_jupiter.camera_location AS camera_location,
  mesa_prod.mesa_lake_prod.image_jupiter.robot_name AS robot_name,
  mesa_prod.mesa_lake_prod.image_jupiter.has_nearby_stop_event AS has_nearby_stop_event,
  mesa_prod.mesa_lake_prod.image_jupiter.operating_field_name AS operating_field_name,
  mesa_prod.mesa_lake_prod.image_jupiter.farm AS farm,
  mesa_prod.mesa_lake_prod.image_jupiter.bag_name AS bag_name,
  mesa_prod.mesa_lake_prod.image_jupiter.closest_object_info__json AS closest_object_info__json,
  mesa_prod.mesa_lake_prod.image_jupiter.collected_on AS collected_on,
  mesa_prod.mesa_lake_prod.image_jupiter.hard_drive_name AS hard_drive_name,
  mesa_prod.mesa_lake_prod.image_jupiter.state AS state,
  mesa_prod.mesa_lake_prod.image_jupiter.special_notes AS special_notes,
  mesa_prod.mesa_lake_prod.image_jupiter.hdr_mode AS hdr_mode,
  mesa_prod.mesa_lake_prod.image_jupiter.group_id AS group_id,
  mesa_prod.mesa_lake_prod.image_jupiter.jdb_s3_path AS jdb_s3_path,
  mesa_prod.mesa_lake_prod.image_jupiter.ros_s3_path AS ros_s3_path,
  mesa_prod.mesa_lake_prod.image_jupiter.bundle AS bundle,
  mesa_prod.mesa_lake_prod.image_jupiter.gps_can_data__json AS gps_can_data__json,
  mesa_prod.mesa_lake_prod.image_jupiter.implement_angle_data__json AS implement_angle_data__json,
  mesa_prod.mesa_lake_prod.image_jupiter.category AS category,
  mesa_prod.mesa_lake_prod.image_jupiter.description AS description,
  mesa_prod.mesa_lake_prod.image_jupiter.autonomy_state__json AS autonomy_state__json,
  mesa_prod.mesa_lake_prod.image_jupiter.sensor_type AS sensor_type,
  mesa_prod.mesa_lake_prod.image_jupiter.geohash AS geohash,
  mesa_prod.mesa_lake_prod.image_jupiter.calibration_data__json AS calibration_data__json,
  mesa_prod.mesa_lake_prod.image_jupiter.tractor_type AS tractor_type
FROM
  mesa_prod.mesa_lake_prod.image_jupiter
WHERE
  hard_drive_name = 'JUPD-0306_2024-2-28'
  AND gps_can_data__json IS NOT NULL
  AND special_notes IS NOT NULL
"""

f"""
  SELECT img.id, img.collected_on
  FROM mesa_prod.mesa_lake_prod.image_jupiter as img
  JOIN mesa_prod.mesa_lake_prod.image_artifact_jupiter AS A
      ON A.image = img.id
  WHERE img.teleop_request__json IS NOT NULL
  AND img.teleop_request__json:original_roi IS NOT NULL
  AND NOT EXISTS(
          SELECT * from mesa_prod.mesa_lake_prod.annotation_jupiter as ann WHERE ann.image = img.id
          AND ann._cls = 'Annotation.CategoricalAnnotation'
          AND ann.is_active_version = true
      )
  LIMIT 10
  ;"""

df = databricks.execute(query)
print(df.shape, len(df)//2)

(45934, 28) 22967


In [11]:
len(df.camera_location.unique())

16

In [ ]:
import ast
root_dir = '/data2/jupiter/datasets/'
dataset = 'halo_rgb_stereo_train_v6_2'
csv = os.path.join(root_dir, dataset, 'master_annotations_dedup.csv')
converters = {"label_map": ast.literal_eval, "label_counts": ast.literal_eval}
# converters = {}
df = pd.read_csv(csv, converters=converters)

ab_df = df[df.has_airborne == True]
print(ab_df.shape, ab_df.iloc[0].geohash)
ab_df["geohash_6"] = ab_df["geohash"].apply(lambda x: x[:6])

from jupiterdata.db import databricks_connector
databricks = databricks_connector.Databricks()

In [15]:
LEFT_CAMERAS = ('T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02')
geohash_6_limit = 500
geohash_6_expand = 1.5
surround_ids = []
for geohash_6 in ab_df.geohash_6.unique():
    collected_on_days = ab_df[(ab_df.geohash_6 == geohash_6)].collected_on_day.unique()
    for collected_on_day in collected_on_days:
        current = len(ab_df[(ab_df.geohash_6 == geohash_6) & (ab_df.collected_on_day == collected_on_day)].unique_id.unique())
        expand = int(geohash_6_limit / current * geohash_6_expand)
        if expand < 50:
            expand = 50
        elif expand > 500:
            expand = 500

        query = f"""
        SELECT
          IMJ.id AS id,
          IMJ.robot_name AS robot_name,
          IMJ.camera_location AS camera_location,
          IMJ.operating_field_name AS operating_field_name,
          IMJ.bag_name AS bag_name,
          IMJ.collected_on AS collected_on,
          IMJ.hard_drive_name AS hard_drive_name,
          IMJ.geohash AS geohash
        FROM
          mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
        WHERE
          collected_on LIKE '{collected_on_day[:10]}%'
          AND geohash LIKE '{geohash_6}%'
          AND camera_location IN {LEFT_CAMERAS}
        LIMIT {expand};
        """
        df = databricks.execute(query)
        surround_ids += df.id.to_list()
        # print(geohash_6, collected_on_day, current, len(df))
    #     break
    # break
    print('queried geohash_6', geohash_6)

len(surround_ids)

queried geohash_6 9xjgd1
queried geohash_6 9xjg99
queried geohash_6 9xjg9c
queried geohash_6 9xjg3g
queried geohash_6 9xjg3x
queried geohash_6 9xjg3z
queried geohash_6 9xjg4x
queried geohash_6 9v5ks7
queried geohash_6 9v5kw6
queried geohash_6 9v5kx5
queried geohash_6 9xjg9r
queried geohash_6 9xjg5n
queried geohash_6 9v5kwg
queried geohash_6 9v5kwu
queried geohash_6 9v5kwy
queried geohash_6 9v5kw7
queried geohash_6 9v5kw5
queried geohash_6 9v5kxn
queried geohash_6 dju21p
queried geohash_6 9xjg93
queried geohash_6 9v5kwv
queried geohash_6 9v5kww
queried geohash_6 9v5kt2
queried geohash_6 9v5kxh
queried geohash_6 9v5ks6
queried geohash_6 dp03h5
queried geohash_6 dp03hh
queried geohash_6 dp03h6
queried geohash_6 dp03h7
queried geohash_6 dp03hj
queried geohash_6 dp03k6
queried geohash_6 dp03k7
queried geohash_6 9xjg3r
queried geohash_6 9xjg92
queried geohash_6 9v5ksk
queried geohash_6 9xjg3f
queried geohash_6 9xjg98
queried geohash_6 9xjg9q
queried geohash_6 9xjgyh
queried geohash_6 dju21n


185045

In [18]:
remain_ids = list(set(surround_ids) - set(df.id.to_list()))
len(remain_ids)

184858

In [19]:
remain_ids[0]

'6568aa0b113f10bb08cf0ed9'

## Create a dataset

In [4]:
# df = pd.read_csv('./dust_threshold/human_vehicle_in_dust_for_train_total.csv')
# df = pd.read_csv('./dust_threshold/dust_productivity.csv')
# df = pd.read_csv('/data/jupiter/datasets/Jupiter_train_v5_11/driveable_not_labeled.csv')
# df = pd.read_csv('/data/jupiter/datasets/Jupiter_train_v5_11/less_than_half_labeled.csv')
# df = pd.read_csv('/data/jupiter/datasets/Jupiter_train_v5_11/less_than_half_or_driveable_not_labeled_for_relabeling.csv')
# df = pd.read_csv('/data/jupiter/li.yu/exps/driveable_terrain_model/rgb_baseline_sample_a_v3_2/20230925_halo_rgb_stereo_train_v3_epoch0/pruned_ids.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/Jupiter_train_v6_2/train_v6_2_categorical_count.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_hard_cases/20231219_20240119_halo_rgb_left_airborne.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/rev1_test_missing_human_pose.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/rev1_test_missing_human_clothing.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/rev1_test_humanv15_day_front_standing_ids.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/train_v8_0_miss_human_pose.csv')
df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/test_v8_0_miss_human_pose.csv')
print(df.shape, 'id' in df, 'unique_id' in df)

(12837, 1) True False


In [5]:
# unlabeled dataset
Dataset.create(
    name="halo_test_v8_0_miss_human_pose_label",
    description="12837 images from test set v8.0, that miss human pose/clothing/occlusion categorical labels",
    kind='image',  # annotation or image
    image_ids=df.id.to_list())

{}

In [7]:
# imageids_to_dataset(
#     image_ids=test_df[test_df.camera_location.str.endswith('left')].id.to_list(),
#     dataset_name="20230823_labeled_right_images_12k_left_images_labels",
#     dataset_description="left images and labels of 20230823_labeled_right_images_12k",
#     dataset_kind='annotation',  # annotation or image
#     mode='mono',  # stereo or mono
#     camera_location=CORE_LEFT_CAMERAS,  # CORE_LEFT_CAMERAS or CORE_RIGHT_CAMERAS or ALL_CORE_CAMERAS
# )
# imageids_to_dataset(
#     image_ids=test_df[test_df.camera_location.str.endswith('right')].id.to_list(),
#     dataset_name="20230823_labeled_right_images_12k_right_images_labels",
#     dataset_description="right images and labels of 20230823_labeled_right_images_12k",
#     dataset_kind='annotation',  # annotation or image
#     mode='mono',  # stereo or mono
#     camera_location=CORE_RIGHT_CAMERAS,  # CORE_LEFT_CAMERAS or CORE_RIGHT_CAMERAS or ALL_CORE_CAMERAS
# )

In [8]:
# # in master branch halo datasets
# annotations = imageids_to_annotation_df(
#     image_ids=df.id.to_list(),
# )

imageids_to_dataset(
    image_ids=test_df.id.to_list(),
    dataset_name="halo_vehicles_driving_through_dust_images_nodust_reserved_stereo",
    dataset_description="labeled vehicles from halo_vehicles_driving_through_dust_images_nodust_reserved",
    dataset_kind='image',  # annotation or image
    mode='stereo',  # stereo or mono
    production_dataset=False,
)
# Dataset.create(
#     name="20230925_halo_rgb_stereo_train_v3_pruned",
#     description="180 images with >0.5 focal loss from dataset 20230925_halo_rgb_stereo_train_v3",
#     kind=Dataset.KIND_ANNOTATION,
#     image_ids=df.id.to_list())

{'UserId': 'AROAWGGSD7MOMRB2MNJIQ:li.yu@bluerivertech.com', 'Account': '425642425116', 'Arn': 'arn:aws:sts::425642425116:assumed-role/AWSReservedSSO_jupiter_prod_engineer_bfbc4d159852cd6d/li.yu@bluerivertech.com', 'ResponseMetadata': {'RequestId': '9f9bb0e3-d745-4ca3-a85b-68f13baa9662', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9f9bb0e3-d745-4ca3-a85b-68f13baa9662', 'content-type': 'text/xml', 'content-length': '506', 'date': 'Wed, 20 Mar 2024 14:17:40 GMT'}, 'RetryAttempts': 0}}
Preparing stereo dataframe for {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03', 'I02': 'I04'}...
Size of left dataframe: 7598
Size of stereo dataframe: 7534
Preparing stereo dataframe for {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15', 'I02': 'I03'}...
Size of left dataframe: 3802
Size of stereo dataframe: 3767
Sending 15100 image ids for creating dataset
Time taken to prepare data for dataset creation job:

## Download a dataset

In [ ]:
# to download large image dataset and save in jpg format, change the following files
# note original files are ending with .bk, and changed files are ending with .save_jpg
# /home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/brtdevkit/data/core/dataset.py
# /home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/brtdevkit/util/aws/s3.py
# /home/li.yu/anaconda3/envs/brtdevkit/lib/python3.9/site-packages/brtdevkit/data/core/dataset.py  # save only one png file

In [11]:
# dataset_name = '20240301_5_million_for_self_supervised_part_0'
dataset_name = 'halo_vehicles_driving_through_dust_images_nodust_reserved_labeled'
dataset_dir = os.path.join('/data2/jupiter/datasets', dataset_name)
# dataset_dir = os.path.join('/data/jupiter/li.yu/data', dataset_name)
os.makedirs(dataset_dir, exist_ok=True)

test_dataset = Dataset.retrieve(name=dataset_name)
test_df = test_dataset.to_dataframe()
test_dataset.download(dataset_dir, df=test_df)

2024-03-15 09:25:52,408 - APIRequestor - ERROR - API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}
ERROR:APIRequestor:API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}
2024-03-15 09:26:00,122 - Dataset - INFO - start downloading dataset: 65f473c0d59c0c794ff17603 into /data2/jupiter/datasets/halo_vehicles_driving_through_dust_images_nodust_reserved_labeled
2024-03-15 09:26:00,122 - Dataset - INFO - start downloading dataset: 65f473c0d59c0c794ff17603 into /data2/jupiter/datasets/halo_vehicles_driving_through_dust_images_nodust_reserved_labeled
INFO:Dataset:start downloading dataset: 65f473c0d59c0c794ff17603 into /data2/jupiter/datasets/halo_vehicles_driving_through_dust_images_nodust_reserved_labeled
/home/li.yu/anaconda3/envs/brtdevkit/lib/python3.9/site-pack

In [2]:
dataset_name = '20240301_5_million_for_self_supervised_part_0'
df = pd.read_csv(os.path.join('/data/jupiter/datasets', dataset_name, 'annotations.csv'))
# df = pd.read_csv(os.path.join('/data/jupiter/li.yu/data', dataset_name, 'annotations.csv'))
print(df.shape)
df.iloc[0].artifact_debayeredrgb_0_save_path

/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (10,20,21,22,23,24,26,27,29,30,31,32,33,34,35,36,46,57,70,84,85,87,91,93,103,106,107,115,132,137,138,140,142,145,146,158,192,193) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(999999, 197)


'images/64bef342438af9b8c7971547.jpg'

In [3]:
files = os.listdir(os.path.join('/data/jupiter/datasets', dataset_name, 'images'))
len(files)

999940

In [4]:
from PIL import Image
from tqdm import tqdm

corrupted = []
for f in tqdm(files, total=len(files)):
    try:
        img = Image.open(os.path.join('/data/jupiter/datasets', dataset_name, 'images', f))
        img.close()
    except:
        corrupted.append(f)
len(corrupted)

100%|██████████| 999940/999940 [18:36<00:00, 895.46it/s]


29

In [22]:
good = list(set(files) - set(corrupted))
good = [f[:-4] for f in good if f.endswith('.jpg')]
print(len(good))

999799


In [23]:
df3 = pd.DataFrame(data={'id': good})
print(df3.shape)
df3.to_csv(os.path.join('/data/jupiter/datasets', dataset_name, 'saved_ids.csv'), index=False)

(999799, 1)
